# Data Mining – Classification Part 2

## Dry Bean Dataset Classification and SVM Kernel Visualizations

**Authors**
### Eli Kaustinen and Gabriel Marcelino

**Dataset Description:**  
This notebook uses the [Dry Bean Dataset](https://archive.ics.uci.edu/dataset/602/dry+bean+dataset) from the UCI Machine Learning Repository. This dataset contains more than 10 attributes (features) and over 10,000 instances (13611 instances) of dry beans, where each instance is characterized by features (such as Area, Perimeter, MajorAxisLength, etc.) and the target variable `Class` (which specifies one of several bean types).

**Problem Statement:**  
For this exercise, we implement three classification methods, and compare the results for each to see which one is more effective:
- k‑Nearest Neighbours (kNN)
- Support Vector Machine (SVM) (using the RBF kernel)
- Random Forest

We will:
1. Load the appropriate packages.
2. Pre-process and subset the data.
3. Split the data into training and testing sets.
4. Build the classification models.
5. Run the models (make predictions).
6. Display quantitative and visual classification results.
7. Provide the confusion matrix for each classifier.
8. Compute accuracy, sensitivity (recall), specificity (and also precision and F1 score for completeness).
9. Explain the use of the ROC curve and the meaning of its area (AUC).
10. Compare the results of the classifiers and discuss any differences.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier



# Set Pandas options to display all columns and a reasonable number of rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# For data preprocessing and splitting
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# For classification models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# For evaluation metrics
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score, 
                             recall_score, f1_score, classification_report)

# For synthetic data (for decision boundary visualization)
from sklearn.datasets import make_moons
from matplotlib.colors import ListedColormap

## Part 1: Classification on the Dry Bean Dataset

We now load and pre-process the Dry Bean dataset. This dataset is a multiclass problem with a target variable called `Class`.

In [4]:
# Load the Dry Bean dataset
# Make sure the file "Dry_Bean_Dataset.csv" is in your working directory.
df = pd.read_csv('Dry_Bean_Dataset.csv')

# Display the first few rows of the dataset
print("Dataset Head:")
print(df.head())

print("\nDataset Info:")
print(df.info())

print("\nMissing Values per Column:")
print(df.isnull().sum())

Dataset Head:
    Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRation  \
0  28395    610.291       208.178117       173.888747      1.197191   
1  28734    638.018       200.524796       182.734419      1.097356   
2  29380    624.110       212.826130       175.931143      1.209713   
3  30008    645.884       210.557999       182.516516      1.153638   
4  30140    620.134       201.847882       190.279279      1.060798   

   Eccentricity  ConvexArea  EquivDiameter    Extent  Solidity  roundness  \
0      0.549812       28715     190.141097  0.763923  0.988856   0.958027   
1      0.411785       29172     191.272751  0.783968  0.984986   0.887034   
2      0.562727       29690     193.410904  0.778113  0.989559   0.947849   
3      0.498616       30724     195.467062  0.782681  0.976696   0.903936   
4      0.333680       30417     195.896503  0.773098  0.990893   0.984877   

   Compactness  ShapeFactor1  ShapeFactor2  ShapeFactor3  ShapeFactor4  Class  
0     0.913358  

### 1.1 Data Preprocessing

- **Handling Missing Values:**  
  We fill missing values only for the numeric columns. This prevents errors when attempting to compute means on non‑numeric data.
  
- **Feature Scaling:**  
  We standardize features as both kNN and SVM are sensitive to feature scale.
  
- **Subsetting the Data:**  
  We assume that all columns except `Class` are features.

In [5]:
# Fill missing values only for numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Separate features and target. (Adjust column names if necessary.)
X = df.drop('Class', axis=1)
y = df['Class']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### 1.2 Split the Data into Training and Testing Sets

We use an 80/20 split and stratify on the target variable to maintain class proportions.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

### 1.3 Build the Classification Models

We implement three classifiers:
1. **k‑Nearest Neighbours (kNN):** with k=5.
2. **Support Vector Machine (SVM):** using the RBF kernel.
3. **Random Forest:** with 100 decision trees in the forest.

#### Model 1: k‑Nearest Neighbours (kNN)

In [7]:
# Build kNN classifier with k=5
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

#### Model 2: Support Vector Machine (SVM)

We build an SVM using an RBF kernel. Note that non-linear kernels may require longer training time on very large datasets, but the Dry Bean dataset is of moderate size.

In [8]:
svm_rbf = SVC(kernel='rbf', probability=True, random_state=42)
svm_rbf.fit(X_train, y_train)
y_pred_svm = svm_rbf.predict(X_test)

#### Model 3: RandomForest
We will build a RandomForestClassifier with the following parameters:
- n_estimators=100: This means the model will use 100 decision trees in the forest.
- random_state=42: This ensures reproducibility of results by setting a fixed random seed.


In [9]:
# Ramdom Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

### 1.4 Evaluate the Models

Because this is a multiclass classification problem, we calculate evaluation metrics using macro‑averaging. The metrics computed include:

- **Accuracy:** Overall correctness.
- **Precision (Macro):** Average precision across classes.
- **Recall/Sensitivity (Macro):** Average recall across classes.
- **F1 Score (Macro):** Harmonic mean of precision and recall.
- **Specificity (Macro):** Average true negative rate across classes (computed using a one‑vs‑rest approach).

The confusion matrix is also displayed.

Below is a function to evaluate and print these metrics.

kNN Model Evaluation:
--- kNN Evaluation ---
Confusion Matrix:
[[232   0  21   0   1   3   8]
 [  0 104   0   0   0   0   0]
 [  8   0 308   0   6   2   2]
 [  0   0   0 646   0  10  53]
 [  0   0   6   6 366   0   8]
 [  2   0   1   7   0 383  13]
 [  2   0   0  51  11   5 458]]

Classification Report:
              precision    recall  f1-score   support

    BARBUNYA       0.95      0.88      0.91       265
      BOMBAY       1.00      1.00      1.00       104
        CALI       0.92      0.94      0.93       326
    DERMASON       0.91      0.91      0.91       709
       HOROZ       0.95      0.95      0.95       386
       SEKER       0.95      0.94      0.95       406
        SIRA       0.85      0.87      0.86       527

    accuracy                           0.92      2723
   macro avg       0.93      0.93      0.93      2723
weighted avg       0.92      0.92      0.92      2723

Accuracy: 0.917
Precision (Macro): 0.932
Recall (Sensitivity, Macro): 0.927
Specificity (Macro): 0

In [23]:

def evaluate_model(y_true, y_pred, model_name='Model'):
    cm = confusion_matrix(y_true, y_pred)
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    
    # For specificity in a multiclass setting, compute one-vs-rest specificity per class and average them.
    cm_sum = cm.sum()
    specificity_list = []
    for i in range(cm.shape[0]):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm_sum - (TP + FN + FP)
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_list.append(specificity)
    specificity_macro = np.mean(specificity_list)
    
    print(f"--- {model_name} Evaluation ---")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision (Macro): {precision:.3f}")
    print(f"Recall (Sensitivity, Macro): {recall:.3f}")
    print(f"Specificity (Macro): {specificity_macro:.3f}")
    print(f"F1 Score (Macro): {f1:.3f}\n")
    
    return cm, accuracy, precision, recall, f1, specificity_macro

print("kNN Model Evaluation:")
cm_knn, acc_knn, prec_knn, rec_knn, f1_knn, spec_knn = evaluate_model(y_test, y_pred_knn, model_name='kNN')

print("SVM (RBF Kernel) Model Evaluation:")
cm_svm, acc_svm, prec_svm, rec_svm, f1_svm, spec_svm = evaluate_model(y_test, y_pred_svm, model_name='SVM_RBF')

print("Random Forest Model Evaluation:")
cm_rf, acc_rf, prec_rf, rec_rf, f1_rf, spec_rf = evaluate_model(y_test, y_pred_rf, model_name='Random_Forest')

# ROC Curve and AUC
# Binarize the labels for multi-class ROC calculation
# ROC Curve and AUC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))
n_classes = y_test_binarized.shape[1]

# Get prediction probabilities
models = {'kNN': knn, 'SVM': svm_rbf, 'Random Forest': rf}
colors = ['b', 'g', 'r']

plt.figure(figsize=(8, 6))

for (name, model), color in zip(models.items(), colors):
    y_score = model.predict_proba(X_test)
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_score[:, i])
        plt.plot(fpr, tpr, color, label=f'{name} (Class {i})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


#### Explanation of ROC Curve and AUC (for Binary Classification)

The **ROC (Receiver Operating Characteristic) curve** is a plot that illustrates the performance of a binary classifier as its discrimination threshold is varied. It plots the **True Positive Rate (Recall)** against the **False Positive Rate (1 - Specificity)** for various thresholds.

- **Area Under the Curve (AUC):**  
  The AUC is a single scalar value that summarizes the ROC curve. An AUC of 1 indicates perfect classification, while an AUC of 0.5 suggests no discriminative power (equivalent to random guessing).




In [21]:
# Compare the performance of the models
results_df = pd.DataFrame({
    'Model': ['kNN', 'SVM (RBF)', 'Random Forest'],
    'Accuracy': [acc_knn, acc_svm, acc_rf],
    'Precision': [prec_knn, prec_svm, prec_rf],
    'Recall': [rec_knn, rec_svm, rec_rf],
    'F1 Score': [f1_knn, f1_svm, f1_rf],
    'Specificity': [spec_knn, spec_svm, spec_rf]
})

print("Model Performance Comparison:")
print(results_df)

Model Performance Comparison:
           Model  Accuracy  Precision    Recall  F1 Score  Specificity
0            kNN  0.917003   0.932266  0.927429  0.929568     0.985296
1      SVM (RBF)  0.922145   0.935937  0.932952  0.934297     0.986217
2  Random Forest  0.921410   0.936130  0.931952  0.933904     0.986020


#### Comparison of Classifiers

           Model           Accuracy   Precision    Recall    F1 Score     Specificity
            kNN            0.917003   0.932266    0.927429   0.929568     0.985296
            SVM (RBF)      0.922145   0.935937    0.932952   0.934297     0.986217
            Random Forest  0.921410   0.936130    0.931952   0.933904     0.986020

- SVM (RBF) has the highest accuracy (92.21%), followed closely by Random Forest (92.14%).
- F1-score is highest for SVM (0.9343), meaning it balances precision and recall well.
- Precision is highest for Random Forest (0.9361), meaning it produces fewer false positives.
- Recall is slightly higher for SVM (0.9330), meaning it captures more true positives.
- Specificity is highest for SVM (0.9862), meaning it minimizes false positives better than the others.
### Verdict: SVM (RBF) slightly outperforms the others in overall performance.  

## Visualizing SVM Decision Boundaries with Different Kernel Functions

To illustrate how different SVM kernels form decision boundaries, we now generate a synthetic 2D dataset (using `make_moons`) and train several SVM models with various kernels:

1. **Linear Kernel:**  
   - Straight-line decision boundaries.
2. **Gaussian RBF Kernel:**  
   - Smooth, flexible boundaries. Gamma controls the tightness.
3. **Polynomial Kernel:**  
   - *Homogeneous (coef0=0):* Boundary passes through the origin.
   - *Inhomogeneous (coef0≠0):* Boundary is shifted.
4. **Sigmoid Kernel:**  
   - Decision boundary similar to logistic regression.

The following code trains and plots the decision boundaries.

In [24]:
# Generate a synthetic 2D dataset for visualization
X_syn, y_syn = make_moons(n_samples=200, noise=0.20, random_state=42)

# Define a mesh grid to plot decision boundaries
x_min, x_max = X_syn[:, 0].min() - 0.5, X_syn[:, 0].max() + 0.5
y_min, y_max = X_syn[:, 1].min() - 0.5, X_syn[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                     np.linspace(y_min, y_max, 500))

# Define SVM kernel settings
kernel_settings = {
    'Linear': {'kernel': 'linear'},
    'RBF': {'kernel': 'rbf', 'gamma': 2},
    'Polynomial (Homogeneous, degree=3)': {'kernel': 'poly', 'degree': 3, 'coef0': 0},
    'Polynomial (Inhomogeneous, degree=3)': {'kernel': 'poly', 'degree': 3, 'coef0': 1}
    }

plt.figure(figsize=(15, 12))
i = 1
for name, params in kernel_settings.items():
    clf = SVC(probability=True, random_state=42, **params)
    clf.fit(X_syn, y_syn)
    
    # Compute decision function for each point on the mesh
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.subplot(3, 2, i)
    plt.contourf(xx, yy, Z, alpha=0.75, levels=np.linspace(Z.min(), Z.max(), 20),
                 cmap=plt.cm.coolwarm)
    plt.contour(xx, yy, Z, colors='k', levels=[0], linestyles=['-'])
    plt.scatter(X_syn[:, 0], X_syn[:, 1], c=y_syn, cmap=ListedColormap(['#FF0000', '#0000FF']),
                edgecolors='k', s=30)
    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
                facecolors='none', edgecolors='yellow', linewidths=1.5, label='Support Vectors')
    plt.title(name)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    i += 1

plt.tight_layout()
plt.show()


## Conclusion

This notebook has demonstrated the complete workflow for a classification exercise using a UCI dataset with more than 10 attributes and 10,000 instances (Dry Bean Dataset). We built classifiers using k‑Nearest Neighbours SVM (with the RBF kernel), and Random Forest. We then evaluated them using confusion matrices and different metrics (accuracy, precision, recall/sensitivity, specificity, and F1 score). Additionally, we provided an explanation of ROC curves (for binary classification) and visualized SVM decision boundaries with various kernels using a synthetic dataset. 

## Algorithm of the Solution

The solution uses three classification algorithms: k-Nearest Neighbours (kNN) with k=5, Support Vector Machine (SVM) with an RBF kernel, and Random Forest with 100 decision trees. These classifiers are applied to the Dry Bean Dataset, where the features are preprocessed, standardized, and split into training and testing sets. Each model is then trained on the training set and evaluated on the test set using various performance metrics like accuracy, precision, recall, F1 score, and specificity.

## Analysis of Findings

The results across the three classifiers—kNN, SVM (RBF), and Random Forest—are very similar, with accuracy values within ~0.5% of each other. SVM (RBF) has the highest accuracy (0.9221) and recall (0.9330), making it slightly better at correctly identifying positive cases, while Random Forest has the highest precision (0.9361), meaning it makes fewer false positive errors. Specificity is also highest for SVM, indicating strong negative case classification. Given these small differences, statistical significance tests like McNemar’s or a paired t-test would be needed to confirm whether the variations are meaningful. If performance remains comparable, factors such as computational efficiency, interpretability, and ease of tuning should guide model selection.

## References

Dry Bean Dataset: UCI Machine Learning Repository. <br>
Scikit-learn documentation: https://scikit-learn.org.


